In [90]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [91]:
# Import the database
all_places = pd.read_csv(r'C:\Users\otrin\OneDrive\Desktop\Git\World_Weather_Analysis\weather_data\WeatherPy_vacation.csv')

In [92]:

# Choosing four cities in Russia that fits the vacation
cities_df = all_places.loc[all_places['Country'] == 'RU']

In [93]:
cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain,Snow
4,51,Ivanteyevka,RU,2020-06-28 16:20:25,55.97,37.92,75.99,60,46,8.95,0.46,0
20,248,Veydelevka,RU,2020-06-28 16:20:32,50.15,38.45,74.44,76,100,6.26,0.40,0
27,321,Mozhaysk,RU,2020-06-28 16:20:35,55.50,36.03,70.86,84,97,4.27,1.15,0
40,484,Kholm-Zhirkovskiy,RU,2020-06-28 16:20:41,55.52,33.47,74.05,76,76,1.43,0.25,0
43,518,Chekhov,RU,2020-06-28 16:20:42,55.14,37.47,73.40,64,98,8.95,1.76,0


In [94]:
cities_df = cities_df.drop([40])

In [95]:
Ivanteyevka = (55.97, 37.92)
Veydelevka= (50.15, 38.45)
Mozhaysk = (55.50, 36.03)
Chekhov = (55.14, 37.47)


In [96]:
#import dependencies
import requests
import gmaps
import gmaps.datasets

#Import API key
from config import g_key

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

#Latitude longtitude pairs
Ivanteyevka = (55.97, 37.92)
Veydelevka= (50.15, 38.45)
Mozhaysk = (55.50, 36.03)
Chekhov = (55.14, 37.47)
fig = gmaps.figure()
itinerary = gmaps.directions_layer(Ivanteyevka, Veydelevka, waypoints=[Mozhaysk, Chekhov], travel_mode='DRIVING')
#locations = cities_df[["Lat", "Lng"]]
fig.add_layer(itinerary)
fig

Figure(layout=FigureLayout(height='420px'))

In [110]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Ivanteyevka,RU,75.99,55.97,37.92,
20,Veydelevka,RU,74.44,50.15,38.45,
27,Mozhaysk,RU,70.86,55.50,36.03,
43,Chekhov,RU,73.40,55.14,37.47,


In [111]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params = {
        "radius": 5000,
        "types": "lodging",
        "key": g_key,
        "location": f"{lat},{lng}"}
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  

In [112]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Ivanteyevka,RU,75.99,55.97,37.92,New Slides
20,Veydelevka,RU,74.44,50.15,38.45,Po Foto
27,Mozhaysk,RU,70.86,55.50,36.03,Country house U Ivanicha
43,Chekhov,RU,73.40,55.14,37.47,Cottage Edu v gosti


In [113]:
len(hotels["results"])

17

In [114]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [115]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [109]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Ivanteyevka,RU,75.99,55.97,37.92,
20,Veydelevka,RU,74.44,50.15,38.45,
27,Mozhaysk,RU,70.86,55.50,36.03,
43,Chekhov,RU,73.40,55.14,37.47,
